In [17]:
from ultralytics import YOLO
import cv2

In [18]:
model = YOLO('D:/projects/football_analysis/models/best_11n.pt')

In [19]:
cap = cv2.VideoCapture('D:/projects/football_analysis/input_videos/08fd33_4.mp4')
frames = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)

In [21]:
type(detection)

list

In [22]:
detection[0].names

{0: 'ball', 1: 'goalkeeper', 2: 'player', 3: 'referee'}

In [20]:
detections = []
detection = model.predict(frames[:20], conf=0.1)
detections += detection
detection2 = model.predict(frames[:20], conf=0.1)
detections += detection2


0: 384x640 1 ball, 1 goalkeeper, 20 players, 4 referees, 53.3ms
1: 384x640 1 ball, 1 goalkeeper, 20 players, 4 referees, 53.3ms
2: 384x640 1 goalkeeper, 22 players, 4 referees, 53.3ms
3: 384x640 1 ball, 1 goalkeeper, 22 players, 4 referees, 53.3ms
4: 384x640 1 ball, 1 goalkeeper, 22 players, 3 referees, 53.3ms
5: 384x640 1 goalkeeper, 23 players, 4 referees, 53.3ms
6: 384x640 1 goalkeeper, 23 players, 4 referees, 53.3ms
7: 384x640 1 ball, 1 goalkeeper, 22 players, 4 referees, 53.3ms
8: 384x640 1 ball, 1 goalkeeper, 20 players, 4 referees, 53.3ms
9: 384x640 1 ball, 20 players, 3 referees, 53.3ms
10: 384x640 1 ball, 20 players, 3 referees, 53.3ms
11: 384x640 19 players, 3 referees, 53.3ms
12: 384x640 20 players, 3 referees, 53.3ms
13: 384x640 1 ball, 20 players, 3 referees, 53.3ms
14: 384x640 1 ball, 21 players, 3 referees, 53.3ms
15: 384x640 23 players, 3 referees, 53.3ms
16: 384x640 21 players, 3 referees, 53.3ms
17: 384x640 1 ball, 21 players, 3 referees, 53.3ms
18: 384x640 1 ball, 2

In [25]:
type(detections)

list

In [26]:
len(detections)

40

In [28]:
detections[1].names

{0: 'ball', 1: 'goalkeeper', 2: 'player', 3: 'referee'}